In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
if(os.path.split(os.getcwd())[1] == "classify-to-learn"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = False

Current Working Directory: progettoVIPM


In [3]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset
from utils.v2 import ExtendedEncoder, Encoder

In [4]:
from utils.loadersAndEnums import datasets, ImageDataset
from torch.utils.data import DataLoader, ConcatDataset, random_split

trainU = ImageDataset(dataset=datasets.TRAINING_UNLABELED,network_input_size=224, cuda=cuda)
unlabeled = DataLoader(trainU,128,True)

In [5]:

device = torch.device('cpu')
#PATH = "Storage\models\AutoEncoderModelV2.pth"
PATH = 'Storage\models\\finetunedResNet50.pth'

if cuda:
    model = torch.load(PATH)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
else:    
    model = torch.load(PATH, map_location=torch.device('cpu') )


In [84]:
all_preds = []
all_out = []
model.eval()
with torch.no_grad():
    for inputs,labels in tqdm(unlabeled):
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        print('outputs: ', outputs)
        print('outputs: ', outputs.shape)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_out.extend(outputs.cpu().numpy())
        
all_preds = np.array(all_preds)
all_out = np.array(all_out)


  0%|          | 1/887 [00:11<2:47:42, 11.36s/it]

outputs:  tensor([[-1.6666, -1.9691, -1.0686,  ..., -4.5402, -3.1432, -2.9515],
        [-3.1942, -3.2076, -3.1805,  ..., -3.7901, -2.7987, -2.1621],
        [-1.9498, -1.0083, -2.7844,  ..., -5.4915, -3.2666, -3.9290],
        ...,
        [-4.1192, -3.3378, -2.9261,  ..., -2.9806, -2.2687, -2.1883],
        [-4.4205, -5.0510, -4.6419,  ..., -2.1618, -4.2665, -5.1066],
        [-2.9069, -3.3245, -2.1991,  ..., -3.6803, -3.3446, -1.8926]])
outputs:  torch.Size([128, 251])


  0%|          | 1/887 [00:12<3:02:46, 12.38s/it]


KeyboardInterrupt: 

In [45]:
all_preds.tofile('all_preds.dat')

In [46]:
all_out.tofile('all_out.dat')

In [89]:
all_outfromfile = np.fromfile('all_out.dat')
print('all_outfromfile.shape: ', all_outfromfile.shape)
print('all_outfromfile: ', all_outfromfile[251*10:251*12])

all_predsfromfile = np.fromfile('all_preds.dat')
print('all_predsfromfile: ', all_predsfromfile)

all_outfromfile.shape:  (14238602,)
all_outfromfile:  [-2.16569939e+01 -1.34433041e+01 -5.17071166e+02 -1.43445950e+01
 -1.38958613e-03 -3.18893957e+00 -1.14185593e+02 -3.86089504e-01
 -1.23228833e-01 -4.21506117e+01 -6.66457397e+02 -4.27430649e+01
 -1.75377922e+01 -1.20305821e-01 -1.83984955e+02 -4.62129090e+02
 -2.63294637e-01 -7.62124390e+02 -2.66907307e-07 -1.67226677e+01
 -5.81665268e+01 -8.95967364e-01 -1.81731992e-02 -5.78819439e-02
 -1.21963242e+02 -2.86823235e+01 -4.62751329e-01 -4.01715468e-03
 -1.09553661e+01 -7.63213486e-02 -1.08947739e-01 -4.27984848e+01
 -1.46961269e+01 -1.37301139e-03 -7.95226289e+01 -6.83934758e-08
 -1.23356857e+02 -5.46397934e+01 -8.97421837e+00 -3.25734176e+01
 -2.52721672e+01  7.24990822e-09 -4.58425507e-02 -1.39261566e+02
 -1.18041592e+01 -1.16705403e-01 -9.53617454e-01 -1.37329922e+01
 -1.09394919e-02 -1.75242475e-03 -2.93954125e+01 -7.38595367e-01
 -5.97236918e+00 -1.25414562e+01 -1.31029644e+01 -3.24816345e+02
 -4.28959885e+01 -1.43961811e+01 -8.

In [90]:
#outputs = torch.from_numpy(all_out[251:251*2])
print('outputs: ', outputs[1])
#print('outputs: ', outputs.shape)
m, predicted = torch.max(outputs, 1)
print('predicted: ', predicted)
print('m: ', m)
all_out=[]
all_out.extend(outputs.cpu().numpy())
all_out = np.array(all_out)
print('all_out: ', all_out)

outputs:  tensor([-3.1942, -3.2076, -3.1805, -1.5229, -5.3111, -3.6859, -3.4493, -1.6922,
        -4.3194, -4.5440, -2.2851, -0.4768, -2.9174, -1.0429, -3.1577, -4.8870,
        -5.0210, -0.8064, -3.2109, -4.6794, -2.8105, -4.8863, -2.8291, -4.4676,
        -2.7008, -3.0495, -3.8444, -2.7970, -3.4362, -3.0425, -2.9252, -2.7762,
        -4.3907, -2.5772, -1.7246, -4.1167, -3.2755, -0.6474, -3.8971, -2.5141,
        -3.1906, -2.9875, -3.6658, -5.1596, -4.3877, -1.6885, -3.6453, -2.8201,
        -3.5226, -3.7529, -4.2427, -4.4593, -3.1527, -0.8813, -3.6429, -2.2493,
        -3.2671,  1.6185, -2.7247, -3.9661, -0.5604, -2.3728, -3.8313, -1.3625,
        -4.8442, -0.0520, -2.7110, -3.1933, -5.9032, -2.7781, -3.8988, -4.1031,
        -2.3454, -4.1806, -3.1994, -5.2528, -4.0405, -2.3643, -3.8101, -1.1431,
        -1.8954, -3.0582, -3.2240, -2.4123, -4.6173, -2.4923, -3.7934, -4.5722,
        -4.2167, -4.7777, -1.5372, -2.4505, -1.3454, -3.4602, -2.3712,  0.2293,
        -1.4169, -2.1820, -2.9

In [96]:
all_outfromfile.shape

(14238602,)

In [98]:
outputs = torch.from_numpy(all_outfromfile[0:251])
print('outputs: ', outputs)
#print('outputs: ', outputs.shape)
m, predicted = torch.max(outputs)
print('predicted: ', predicted)
print('m: ', m)


outputs:  tensor([-8.5639e+00, -6.1804e+01, -5.2439e+00, -1.1161e+01, -2.4762e+01,
        -1.1361e+01, -9.9256e-01, -1.4660e+02, -3.3844e+02, -1.0753e+01,
        -4.8939e+02, -1.1471e+03, -1.3493e+01, -6.4822e+02, -1.3594e+01,
        -1.6202e+00, -2.2923e-05, -8.7862e+02, -5.7944e+00, -7.9437e+02,
        -1.3284e+03, -5.0117e+02, -2.4273e+02, -8.3528e+01, -2.1466e+02,
        -6.5785e+02, -3.5622e+01, -2.4153e-01, -4.2551e+01, -4.6735e+01,
        -1.6561e+02, -8.7775e+00, -3.8349e-01, -6.8733e+02, -3.2666e+01,
        -3.3960e+01, -1.0665e+02, -1.4136e+03, -6.2657e+02, -9.1366e+02,
        -1.0020e+02, -1.7848e+00, -2.7969e-10, -4.9034e+02, -2.3761e+01,
        -1.0563e+03, -2.3547e+03, -9.4195e+02, -1.7182e+02, -3.2199e+00,
        -1.6084e-01,  4.4331e-09, -1.0207e+01, -2.4364e-04, -1.1188e+02,
        -6.3516e+02, -1.6660e+01, -1.3120e-04, -6.3735e+00, -1.5364e-01,
        -2.4954e+00, -1.3239e+03, -3.4160e-02, -6.3938e+00, -1.2581e+01,
        -6.0570e+01, -5.9797e+02, -1.6269

TypeError: iteration over a 0-d tensor